### Phần 1. Giới thiệu và Cài đặt

In [ ]:
# cài đặt spacy
!pip install -U spacy

In [2]:
# Tải về mô hình tiếng Anh (kích thước trung bình, có đủ thông tin cho parsing)
!spacy download en_core_web_md

     ---------------------------------------- 0.0/33.5 MB ? eta -:--:--
     ---------------------------------------- 0.3/33.5 MB ? eta -:--:--
     - -------------------------------------- 1.6/33.5 MB 7.0 MB/s eta 0:00:05
     - -------------------------------------- 1.6/33.5 MB 7.0 MB/s eta 0:00:05
     - -------------------------------------- 1.6/33.5 MB 7.0 MB/s eta 0:00:05
     - -------------------------------------- 1.6/33.5 MB 7.0 MB/s eta 0:00:05
     - -------------------------------------- 1.6/33.5 MB 7.0 MB/s eta 0:00:05
     - -------------------------------------- 1.6/33.5 MB 7.0 MB/s eta 0:00:05
     - -------------------------------------- 1.6/33.5 MB 7.0 MB/s eta 0:00:05
     -- ------------------------------------ 1.8/33.5 MB 986.7 kB/s eta 0:00:33
     --- ------------------------------------ 2.6/33.5 MB 1.3 MB/s eta 0:00:24
     --- ------------------------------------ 2.9/33.5 MB 1.3 MB/s eta 0:00:24
     --- ------------------------------------ 2.9/33.5 MB 1.3 MB/

### Phần 2. Phân tích câu và trực quan hoá

#### 2.1. Tải mô hình và phân tích câu

In [1]:
import spacy

# Tải mô hình tiếng Anh đã cài đặt
# Sử dụng en_core_web_md vì nó chứa các vector từ và cây cú pháp đầy đủ
nlp = spacy.load("en_core_web_md")
# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."
# Phân tích câu với pipeline của spaCy
doc = nlp(text)

#### 2.2. Trực quan hoá cây phụ thuộc

In [ ]:
from spacy import displacy

displacy.serve(doc, style='dep')

d:\miniconda3\envs\nlp2025\lib\site-packages\spacy\displacy\__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [02/Dec/2025 14:39:55] "GET / HTTP/1.1" 200 7538
127.0.0.1 - - [02/Dec/2025 14:39:56] "GET /favicon.ico HTTP/1.1" 200 7538


**Câu hỏi:**
1. **Từ nào là từ gốc (ROOT) của cây? Trả lời:** `jumps`
2. **`jumps` có những từ phụ thuộc nào? Các quan hệ đó là gì?**
    Các từ phụ thuộc của `jumps`:
    - `fox`: Chủ ngữ dạng danh từ
    - `over`: Giới từ bổ nghĩa
3. **`fox` là head của những từ nào? Trả lời:** `The`, `quick`, `brown`

### Phần 3. Truy cập các thành phần trong cây phụ thuộc

In [9]:
text = 'Apple is looking at buying U.K. startup for $1 billions'
doc = nlp(text)

print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-"*70)

for token in doc:
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {list(token.children)}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | [Apple, is, at]
at           | prep       | looking      | VERB     | [buying]
buying       | pcomp      | at           | ADP      | [startup]
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | [U.K., for]
for          | prep       | startup      | NOUN     | [billions]
$            | quantmod   | billions     | NOUN     | []
1            | compound   | billions     | NOUN     | []
billions     | pobj       | for          | ADP      | [$, 1]


### Phần 4. Duyệt cây phụ thuộc để trích xuất thông tin

#### 4.1. Bài toán: Tìm chủ ngữ và tân ngữ của một động từ

In [6]:
text = 'The cat chased the mouse and the dog watched them.'
doc = nlp(text)

for token in doc:
    if token.pos_ == 'VERB':
        verb = token.text
        subject = ''
        obj = ''

        for child in token.children:
            if child.dep_ == 'nsubj':
                subject = child.text
            if child.dep_ == 'dobj':
                obj = child.text

        if subject and obj:
            print(f'Found Triplet: ({subject}, {verb}, {obj})')

Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


#### 4.2. Bài toán: Tìm các tính từ bổ nghĩa cho một danh từ

In [8]:
text = 'The big, fluffy white cat is sleeping on the warm mat.'
doc = nlp(text)

for token in doc:
    if token.pos_ == 'NOUN':
        adjectives = []
        for child in token.children:
            if child.dep_ == 'amod':
                adjectives.append(child.text)

        if adjectives:
            print(f"Danh từ: '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")

Danh từ: 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy', 'white']
Danh từ: 'mat' được bổ nghĩa bởi các tính từ: ['warm']


### Phần 5. Bài tập tự luyện

#### Bài 1. Tìm động từ chính của câu

In [46]:
def find_main_verb(doc):
    for token in doc:
        if token.dep_ == 'ROOT':
            print(f"Động từ chính trong câu: {token.text}")
            return
    print("Không tìm thấy động từ chính trong câu.")

text = 'The big, fluffy white cat is sleeping on the warm mat.'
doc = nlp(text)
find_main_verb(doc)

Động từ chính trong câu: sleeping


#### Bài 2. Trích xuất các cụm danh từ (Noun chunks)

In [47]:
def get_noun_chunks(doc):
    for token in doc:
        if token.pos_ == 'NOUN':
            chunk = []
            for child in token.children:
                if child.dep_ in ['amod', 'compound', 'det']:
                    chunk.append(child.text)
            chunk.append(token.text)

            if token.head.dep_ in ['det', 'amod', 'compound']:
                chunk = chunk.append(token.head.text)
            
            print(f'Cụm danh từ: {chunk}')

text = 'The big, fluffy white cat is sleeping on the warm mat.'
doc = nlp(text)
get_noun_chunks(doc)

Cụm danh từ: ['The', 'big', 'fluffy', 'white', 'cat']
Cụm danh từ: ['the', 'warm', 'mat']


#### Bài 3. Tìm đường đi ngẵn nhất

In [48]:
def get_path_to_root(token, visited):
    # nếu là node cuối
    if token.dep_ == 'ROOT':
        return [token.text]
    
    # tìm đường đi qua các node con
    if token.children:
        for child in token.children:
            if not child in visited:
                visited.add(child)
                path_to_root = get_path_to_root(child, visited)
                if path_to_root is not None:
                    return [token.text] + path_to_root
    
    # tìm đường đi qua node cha
    if token.head and not token.head in visited:
        visited.add(token.head)
        path_to_root = get_path_to_root(token.head, visited)
        if path_to_root is not None:
            return [token.text] + path_to_root

    # trả về None không tìm thấy root
    return None

visited = set()
text = 'The big, fluffy white cat is sleeping on the warm mat.'
doc = nlp(text)

start_idx = 3
print(f'Start token: {doc[start_idx]}')
print(get_path_to_root(doc[start_idx], visited))

Start token: fluffy
['fluffy', 'cat', 'sleeping']


In [28]:
from spacy import displacy
displacy.serve(doc, style='dep')

d:\miniconda3\envs\nlp2025\lib\site-packages\spacy\displacy\__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
